# REMEMBER TO CHANGE THE RUNTIME TO GPU!!

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
from PIL import Image

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

DATA_DIR = '/kaggle/input/plant-pathology-2020-fgvc7'
SEED = 1279
RUN_TRAIN = True

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# !pip install ImageHash

In [3]:
# train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
# train_df.rename(columns={'multiple_diseases': 'multi'}, inplace=True)
# to_drop = ['Train_379', 'Train_782', 'Train_1661'] # dropping repeated/mislabeled samples
# train_df = train_df[~train_df.image_id.isin(to_drop)]
# test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
# print(train_df.head())
# print(test_df.head())
# print(type(train_df.iloc[0]))
# for iterable in train_df.iloc[0].index:
#     print(iterable)

In [4]:
# image display stuff

def disp(img_row):
    img = Image.open(f"{DATA_DIR}/images/{img_row['image_id']}.jpg")
    plt.figure()
    plt.axis('off')
    plt.title(', '.join([f'{lbl}={img_row[lbl]}' for lbl in img_row.index]))
    plt.imshow(img)

In [5]:
# rusty_leaves = train_df[train_df['rust'] > 0]
# idxs = np.random.choice(len(rusty_leaves), size=10)
# for idx in idxs:
#     disp(rusty_leaves.iloc[idx])

In [6]:
# scabby = train_df[train_df['scab'] > 0]
# idxs = np.random.choice(len(scabby), size=10)
# for idx in idxs:
#     disp(scabby.iloc[idx])

In [7]:
# helth = train_df[train_df['healthy'] > 0]
# idxs = np.random.choice(len(helth), size=10)
# for idx in idxs:
#     disp(helth.iloc[idx])

In [8]:
# idxs = np.random.choice(len(test_df), size=10)
# for i in idxs:
#     disp(test_df.iloc[i])

In [9]:
# # looking for similar images (https://github.com/JohannesBuchner/imagehash/blob/master/find_similar_images.py)
# import imagehash
# def find_similar_images(userpaths, hashfunc=imagehash.phash):
#     def is_image(filename):
#         f = filename.lower()
#         return f.endswith(".png") or f.endswith(".jpg") or \
#             f.endswith(".jpeg") or f.endswith(".bmp") or \
#             f.endswith(".gif") or '.jpg' in f or  f.endswith(".svg")
    
#     image_filenames = []
#     for userpath in userpaths:
#         image_filenames += [os.path.join(userpath, path) for path in os.listdir(userpath) if is_image(path)]
#     images = {}
#     problematic = []
#     print("filenames obtained")
#     for img in tqdm(sorted(image_filenames)):
#         try:
#             hash = hashfunc(Image.open(img))
#         except Exception as e:
#             print('Problem:', e, 'with', img)
#             continue
#         if hash in images:
#             print(img, '  already exists as', ' '.join(images[hash]))
#             problematic += [img, *images[hash]]
# #             if 'dupPictures' in img:
# #                 print('rm -v', img)
#         images[hash] = images.get(hash, []) + [img]
#     return problematic
    
# problematic = find_similar_images([f'{DATA_DIR}/images/'])

In [10]:
# # print(problematic)
# pairs = []
# for k in range(0,len(problematic),2):
#     pairs.append( (problematic[k].rsplit('/', 1)[-1], problematic[k+1].rsplit('/', 1)[-1]) )
# print(pairs[-3:])

In [11]:
# idxs = [379, 1173, 782, 592, 815, 1661]
# for i in idxs:
#     disp(train_df.iloc[i])
    
# # seems like we should drop Train_379, Train_782, and Train_1661

In [12]:
# for col in ['healthy', 'multi', 'rust', 'scab']:
#     print(f"{col}, {sum(train_df[col])}")
    
# # data distribution: fairly even except for less multi

## Model Training
Note: This is designed to be runnable even if the data analysis part is not run. As such, there's some code repeat.

In [13]:
from torch.utils.data.dataset import Dataset, Subset
from torch.utils.data import DataLoader
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from sklearn.metrics import roc_auc_score

### Utility Functions

In [14]:
class_names = ['healthy', 'multi', 'rust', 'scab']

In [15]:
def get_train_data():
    train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
    train_df.rename(columns={'multiple_diseases': 'multi'}, inplace=True)
    to_drop = ['Train_379', 'Train_782', 'Train_1661'] # dropping repeated/mislabeled samples
    train_df = train_df[~train_df.image_id.isin(to_drop)]
    files = [f"{DATA_DIR}/images/{fname}.jpg" for fname in train_df['image_id']]
    labels = train_df.iloc[:, 1:].to_numpy().argmax(axis=1).astype(int)
    return files, labels

In [16]:
def get_test_data():
    test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
    files = [f"{DATA_DIR}/images/{fname}.jpg" for fname in test_df['image_id']]
    return files, None

In [17]:
def get_transforms(train=True):
    transforms = [
            torchvision.transforms.Resize((224, 224)), # change this to some variable later!
    ]
    if train:
        transforms += [
            torchvision.transforms.RandomHorizontalFlip(p=0.5),
            torchvision.transforms.RandomVerticalFlip(p=0.5),
            torchvision.transforms.RandomRotation(45, resample=False, expand=False, center=None, fill=None),
            torchvision.transforms.ColorJitter(brightness=.2)
        ]
    transforms.append(torchvision.transforms.ToTensor())
    return torchvision.transforms.Compose(transforms)

In [18]:
def compute_acc(outputs, labels):
    return np.mean(outputs==labels)

def compute_mean_auc(outputs, labels):
    return roc_auc_score(labels, outputs, average='macro', multi_class='ovr')

### Dataset and Dataloader setup

In [19]:
class AppleDataset(Dataset):
    def __init__(self, filepaths, transforms, labels=None):
        self.transforms = transforms
        self.filepaths = filepaths
        self.labels = labels
        
    def __len__(self):
        return len(self.filepaths)
    
    def __getitem__(self, i):
        img = Image.open(self.filepaths[i]).convert('RGB')
        img = self.transforms(img)
        lbl = self.labels[i] if self.labels is not None else None
        return (img, lbl)

In [20]:
# Lightning folds !
class AppleKFoldDataModule(pl.LightningDataModule):
    def __init__(self, k_folds=5):
        self.k_folds = k_folds
        super().__init__()
    
    def setup(self, stage=None):
        train_files, train_labels = get_train_data()
        test_files = get_test_data()
        self.train_dset = AppleDataset(train_files, get_transforms(train=True), labels=train_labels)
        self.val_dset = AppleDataset(train_files, get_transforms(train=False), labels=train_labels)
        self.test_dset = AppleDataset(test_files, get_transforms(train=False))
    
    def setup_folds(self):
        rng = np.random.default_rng(seed=SEED)
        shuffled_idx = rng.permutation(len(self.train_dset))
        self.splits = np.array_split(shuffled_idx, self.k_folds)
        
    def setup_fold_index(self, i):
        train_idx = np.concatenate(self.splits[:i] + self.splits[i+1:])
        val_idx = self.splits[i]
        self.train_fold = Subset(self.train_dset, train_idx)
        self.val_fold = Subset(self.val_dset, val_idx)
    
    def train_dataloader(self, bsz=32):
        return DataLoader(self.train_fold, batch_size=bsz, shuffle=True) # change batch size for variable
    
    def val_dataloader(self, bsz=32):
        return DataLoader(self.val_fold, batch_size=bsz, shuffle=False)
        
    def test_dataloader(self, bsz=32):
        return DataLoader(self.test_dset, batch_size=bsz, shuffle=False)

### Model(s) Setup

In [21]:
def initialize_base_model(model_name, pretrain=True):
    if model_name == 'resnet-50':
        model = torchvision.models.resnet50(pretrained=pretrain)
        inp_size = 224
        out_size = 1000
        
    return model, inp_size, out_size

In [50]:
class GenericAppleArch(nn.Module):
    def __init__(self, base_model_name='resnet-50', out_classes=4):
        super(GenericAppleArch, self).__init__()
        self.ft_extractor, self.inp_size, out_size = initialize_base_model(base_model_name, pretrain=True)
        self.clfr = nn.Sequential(nn.Linear(out_size, out_classes), nn.Softmax(dim=1))
        
    def forward(self, x):
        x = self.ft_extractor(x)
        out = self.clfr(x)
        return out
    

class AppleModel(pl.LightningModule): #Lightning-fied
    def __init__(self, base_model_name='resnet-50', out_classes=4, lr=1e-5, err_backprop_weight = 1.25):
        super().__init__()
        self.model = GenericAppleArch(base_model_name=base_model_name, out_classes=out_classes)
        self.lr = lr
        self.err_backprop_weight = err_backprop_weight
        self.save_hyperparameters()
        
    def on_train_epoch_start(self):
        self.train_outputs = []
        self.train_ys = []
    
    def on_validation_epoch_start(self):
        self.val_outputs = []
        self.val_ys = []
        
    def training_step(self, batch, batch_idx):
        x,y = batch
        out = self.model(x)
        preds = torch.argmax(out, axis=1)
        loss = torch.mean(F.nll_loss(out, y, reduction ='none') * ((preds==y) + self.err_backprop_weight*(preds!=y)))
        self.train_outputs += list(out.detach().cpu().numpy())
        self.train_ys += list(y.detach().cpu())
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x,y = batch
        out = self.model(x)
        self.val_outputs += list(out.detach().cpu().numpy())
        self.val_ys += list(y.detach().cpu().numpy())
        loss = F.nll_loss(out, y)
        self.log('val_loss', loss)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)
        return optimizer
    
    def on_train_epoch_end(self):
        all_outputs = np.array(self.train_outputs)
        all_preds = np.argmax(all_outputs, axis=1)
        all_ys = np.array(self.train_ys)
        epoch_acc = compute_acc(all_preds, all_ys)
        mean_auc = compute_mean_auc(all_outputs, all_ys)
        self.log_dict({'train_acc': epoch_acc, 'train_mean_auc': mean_auc})
        print(f"Train acc={epoch_acc}")
        print(f"Train mean auc={mean_auc}")
    
    def on_validation_epoch_end(self):
        all_outputs = np.array(self.val_outputs)
        all_preds = np.argmax(all_outputs, axis=1)
        all_ys = np.array(self.val_ys)
        epoch_acc = compute_acc(all_preds, all_ys)
        mean_auc = compute_mean_auc(all_outputs, all_ys)
        self.log_dict({'val_acc': epoch_acc, 'val_mean_auc': mean_auc})
        print(f"Val acc={epoch_acc}")
        print(f"Val mean auc={mean_auc}")

### Training Loop

In [51]:
from copy import deepcopy
from datetime import datetime
from pytz import timezone
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint

In [56]:
## WANDB
# https://wandb.ai/manan-goel/MNIST/reports/How-to-Integrate-PyTorch-Lightning-with-Weights-Biases--VmlldzoxNjg1ODQ1
if RUN_TRAIN:
    import wandb
    from pytorch_lightning.loggers import WandbLogger
    from kaggle_secrets import UserSecretsClient
    import os
    
    user_secrets = UserSecretsClient()
    os.environ["WANDB_API_KEY"] = user_secrets.get_secret("wandbKey")

In [57]:
if RUN_TRAIN:
    pl.seed_everything(SEED)
    exp_name = "baseline-augmented"
    try:
        wandb.finish()
    except:
        pass
    k_models = []
    config = {
        'trainer_kwargs': 
            {
                'max_epochs': 30,
                'accelerator': "gpu" if torch.cuda.is_available() else "cpu",
                'default_root_dir': "/kaggle/working/",
                'auto_lr_find': False,
                'log_every_n_steps': 10,
            },
        'model_kwargs':
            {
                'base_model_name': 'resnet-50', 
                'out_classes': 4,
                'lr': 1e-5,
                'err_backprop_weight': 1.25,
            }
    }
    dm = AppleKFoldDataModule(k_folds=5)
    dm.prepare_data()
    dm.setup()
    dm.setup_folds()
    model = AppleModel()
    base_state_dict = model.state_dict()
    print('starting model')
    for k in range(dm.k_folds):
        time = datetime.now(timezone('US/Pacific')).strftime("%m_%d-h%H_m%M")
        save_filepath = f"{exp_name}-fold{k+1}of{dm.k_folds}-{time}"
        config['trainer_kwargs']['logger'] = WandbLogger(project="apple-kaggle", entity="waggle", name=f"{exp_name}_fold{k+1}of{dm.k_folds}")
        config['trainer_kwargs']['callbacks'] = [
            EarlyStopping(monitor="val_loss", mode="min"),
            ModelCheckpoint(dirpath="/kaggle/working/", save_top_k=1, monitor="val_loss", filename=save_filepath)
        ]
        trainer = pl.Trainer(**config['trainer_kwargs'])
        dm.setup_fold_index(k)
        model.load_state_dict(base_state_dict)
        trainer.fit(model, datamodule=dm)
        wandb.finish()

starting model
Problem at: /opt/conda/lib/python3.7/site-packages/pytorch_lightning/loggers/wandb.py 357 experiment


UsageError: Error communicating with wandb process
try: wandb.init(settings=wandb.Settings(start_method='fork'))
or:  wandb.init(settings=wandb.Settings(start_method='thread'))
For more info see: https://docs.wandb.ai/library/init#init-start-error

In [ ]:
if RUN_TRAIN:
    wandb.finish()

## Model Analysis
Testing results of completed models

In [27]:
import numpy as np
from tqdm import tqdm, trange
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image


In [ ]:
# input_model_name = "baseline-fold4of5-06_18-h10_m19.ckpt"
input_model_name = '../input/v65apple/baseline-fold4of5-06_18-h10_m19.ckpt'
# model_path = f"/kaggle/input/appleanalysis/{input_model_name}"
model_path = "/kaggle/input/v65apple/baseline-fold4of5-06_18-h10_m19.ckpt"
ckpt = torch.load(model_path)
# old_model = AppleModel(**ckpt['hyper_parameters'])
old_model = AppleModel.load_from_checkpoint(model_path)

In [ ]:
pl.seed_everything(SEED)
dm = AppleKFoldDataModule(k_folds=5)
dm.prepare_data()
dm.setup()
dm.setup_folds()
dm.setup_fold_index(3)
dl = dm.val_dataloader()

# comment out below if you only want to get the model
old_model.eval()
old_model.on_validation_epoch_start()
val_inputs = []
for i, (x,y) in enumerate(tqdm(dl)):
#     x = x.cuda()
#     y = y.cuda()
    old_model.validation_step((x,y), i)
    val_inputs.append(x)

In [ ]:
class DotDict:
    def __init__(self, **kwargs):
        for k,v in kwargs.items():
            self.__setattr__(k, v)
            
    def __str__(self):
        return str(self.__dict__)

class ValOutputs:
    def __init__(self, inputs, outputs, preds, labels):
        assert len(inputs) == len(outputs) == len(preds) == len(labels)
        self.inputs = inputs
        self.outputs = outputs
        self.preds = preds
        self.labels = labels
        self.indexer = np.arange(len(inputs))
    
    def reindex(self, idxs):
        self.indexer = np.array(idxs)
    
    def reset_index(self):
        self.indexer = range(len(self.inputs))
    
    def __getitem__(self, i):
        idx = self.indexer[i]
        return DotDict(
            inp = self.inputs[idx],
            out = self.outputs[idx],
            pred = self.preds[idx],
            lbl = self.labels[idx]
        )

In [ ]:
outputs = np.array(old_model.val_outputs)
val_out = ValOutputs(
    inputs = torch.concat(val_inputs),
    outputs = outputs,
    preds = np.argmax(outputs,axis=1),
    labels = np.array(old_model.val_ys)
)
print(val_out[0])

In [ ]:
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
train_df.rename(columns={'multiple_diseases': 'multi'}, inplace=True)
to_drop = ['Train_379', 'Train_782', 'Train_1661'] # dropping repeated/mislabeled samples
train_df = train_df[~train_df.image_id.isin(to_drop)]

In [ ]:
val_df = train_df.iloc[dm.val_fold.indices]

# val_outputs = np.array(old_model.val_outputs)
val_df.loc[:,'pred_healthy'] = val_out.outputs[:,0].round(5)
val_df.loc[:,'pred_multi'] = val_out.outputs[:,1].round(5)
val_df.loc[:,'pred_rust'] = val_out.outputs[:,2].round(5)
val_df.loc[:,'pred_scab'] = val_out.outputs[:,3].round(5)
val_df.head()

In [ ]:
i = 0
error = 0
error_indicies = []
while i < len(val_out.labels) and error < 5:#20:
    if val_out.labels[i] != np.argmax(val_out.outputs[i]):
        disp(val_df.iloc[i])
        error_indicies.append(i)
        error += 1
        # :)
    i += 1


## GRADCAM ANALYSIS

In [ ]:
import sys
!pip install ttach
# !git clone https://github.com/jacobgil/pytorch-grad-cam
sys.path.append("../input/gradcam/")
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

In [ ]:
cam_samps = np.where(val_out.preds != val_out.labels)[0] #np.arange(19)
val_out.reindex(cam_samps)

In [ ]:
target_layers = [old_model.model.ft_extractor.layer4[-1]]
input_tensor = val_out[:].inp
print(input_tensor.shape)# Create an input tensor image for your model..
# Note: input_tensor can be a batch tensor with several images!
# images = inputs[cam_samps].permute(0,2,3,1).numpy()

# Construct the CAM object once, and then re-use it on many images:
class_maps = []
with GradCAM(model=old_model.model, target_layers=target_layers, use_cuda=True) as cam:#args.use_cuda)

    # You can also use it within a with statement, to make sure it is freed,
    # In case you need to re-create it inside an outer loop:
    # with GradCAM(model=model, target_layers=target_layers, use_cuda=args.use_cuda) as cam:
    #   ...

    # We have to specify the target we want to generate
    # the Class Activation Maps for.
    # If targets is None, the highest scoring category
    # will be used for every image in the batch.
    # Here we use ClassifierOutputTarget, but you can define your own custom targets
    # That are, for example, combinations of categories, or specific outputs in a non standard model.
    for i in range(len(class_names)):
        tgts = [ClassifierOutputTarget(i)]*len(input_tensor)
        grayscale_maps = cam(input_tensor=input_tensor, targets=tgts)
        class_maps.append(grayscale_maps)
print(class_maps[2].shape)

In [ ]:
for k in range(len(cam_samps)): # for all errors
    fig, ax = plt.subplots(ncols=5, figsize=(25,5))
    cur_val = val_out[k]
    raw_image = cur_val.inp.permute(1,2,0).numpy()
    for i, cn in enumerate(class_names):
        ax[i].imshow( show_cam_on_image(raw_image, class_maps[i][k,:], use_rgb=True) )
        ax[i].axis('off')
        ax[i].set_title(cn)
    ax[-1].imshow( raw_image )
    ax[-1].set_title(f"lbl={class_names[cur_val.lbl]}, pred={class_names[cur_val.pred]}")
    ax[-1].axis('off')
    fig.set_facecolor("#ffb2ab" if cur_val.lbl != cur_val.pred else "#bdffc6")
    fig.show()

In [ ]:
print(val_out.indexer)

In [ ]:
plt.imshow(val_out.inputs[0].permute(1,2,0).numpy())

In [ ]:
error_idx = np.where(val_out.preds != val_out.labels)[0]
plt.hist(val_out.labels[error_idx])
print(class_names)